In [ ]:
matplotlib.rcParams['font.sans-serif'] = "Arial"
sns.set(style="whitegrid", font_scale=1, color_codes=True, palette="colorblind", rc={"grid.linestyle": "dotted", "axes.linewidth": 0.6})

In [ ]:
# Import siganling data
X = preprocessing(AXLm_ErlAF154=True, Vfilter=True, FCfilter=True, log2T=True, mc_row=True)
d = X.select_dtypes(include=['float64']).T
i = X.select_dtypes(include=['object'])

In [ ]:
# Fit DDMC
ddmc = MassSpecClustering(i, n_components=5, SeqWeight=2, distance_method="PAM250", random_state=2).fit(d)
centers = ddmc.transform()

In [ ]:
# Import phenotypes
cv = import_phenotype_data(phenotype="Cell Viability")
red = import_phenotype_data(phenotype="Cell Death")
sw = import_phenotype_data(phenotype="Migration")
c = import_phenotype_data(phenotype="Island")
y = formatPhenotypesForModeling(cv, red, sw, c)
y = y[y["Treatment"] == "A/E"].drop("Treatment", axis=1).set_index("Lines")

In [ ]:
# Centers
_, ax = plt.subplots(1, 4, figsize=(20, 5))
plotCenters_together(ddmc, X, ax[0])

# Scores & Loadings
lines = ["WT", "KO", "KD", "KI", "Y634F", "Y643F", "Y698F", "Y726F", "Y750F ", "Y821F"]
plsr = PLSRegression(n_components=4)
plotScoresLoadings(ax[1:3], plsr.fit(centers, y), centers, y, ddmc.n_components, lines, pcX=1, pcY=2)

# Plot upstream kinases heatmap
plotDistanceToUpstreamKinase(ddmc, [1, 2, 3, 4, 5], ax[-1], num_hits=4, PsP_background=False)

In [ ]:
for ii in range(5):
    cl_idx = [i for i in range(len(labels)) if labels[i] == ii + 1]
    print(np.mean(ddmc.scores_[cl_idx, ii]))

In [ ]:
import numpy as np
pd.DataFrame(ddmc.scores_).mean()

In [ ]:
# Predictions
Xs, models = ComputeCenters(X, d, i, ddmc, 5)
Xs.append(centers)
models.append("DDMC mix")

_, ax = plt.subplots(1, 1)
plotStripActualVsPred(ax, [3, 4, 2, 3, 4], Xs, y, models)

In [ ]:
# Scores & Loadings
lines = ["WT", "KO", "KD", "KI", "Y634F", "Y643F", "Y698F", "Y726F", "Y750F ", "Y821F"]
plsr = PLSRegression(n_components=4)

_, ax = plt.subplots(1, 2, figsize=(10, 5))
plotScoresLoadings(ax, plsr.fit(centers, y), centers, y, ddmc.n_components, lines, pcX=1, pcY=2)

In [ ]:
# Plot upstream kinases heatmap
_, ax = plt.subplots(1, 1)
plotDistanceToUpstreamKinase(ddmc, [1, 2, 3, 4, 5], ax, num_hits=10)